In [1]:
!pip install gradio transformers torch accelerate huggingface-hub PyMuPDF fpdf --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 43.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "ibm-granite/granite-3.3-2b-instruct"
hf_token = "hf_..................."  # your token

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    device_map="auto",
    torch_dtype=torch.float16
)

print("✅ IBM Granite model loaded successfully!")


tokenizer_config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

✅ IBM Granite model loaded successfully!


In [5]:
# Features

import fitz  # for PDF
from fpdf import FPDF

chat_memory = []

# 🔸 Chat Assistant
def smart_city_chat(query):
    chat_memory.append(f"🧑: {query}")
    inputs = tokenizer(query, return_tensors="pt").to("cuda")
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=200)
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)
    chat_memory.append(f"🤖: {reply}")
    return "\n".join(chat_memory)

# 🔸 PDF Policy Summarizer
def summarize_pdf(file):
    doc = fitz.open(stream=file.read(), filetype="pdf")
    text = "".join(page.get_text() for page in doc)
    prompt = f"Summarize the following city policy document:\n{text}"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.inference_mode():
        output = model.generate(**inputs, max_new_tokens=400)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 🔸 KPI Summary Card
def kpi_summary():
    return """
    ### 🌆 Smart City KPIs
    - 🏙️ Air Quality Index: **72 (Moderate)**
    - 💧 Water Usage: **120 L/person/day**
    - ♻️ Waste Recycled: **58%**
    """

# 🔸 Eco Tips Generator
import random

def generate_eco_tips():
    prompts = [
        "List exactly 3 simple eco-friendly habits for daily office life.",
        "List exactly 3 easy ways to reduce electricity usage at home.",
        "List exactly 3 quick water-saving tips for city residents.",
        "List exactly 3 simple green travel habits for working people.",
        "List exactly 3 ways to cut down on plastic use.",
        "List exactly 3 green actions during lunch breaks.",
        "List exactly 3 things a person can do daily to help the planet.",
        "List exactly 3 ways to be sustainable while working from home.",
        "List exactly 3 energy-saving tips for using air conditioning.",
        "List exactly 3 smart city tips to live a greener life."
    ]

    prompt = random.choice(prompts) + " Keep each tip under 20 words."

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.inference_mode():
        output = model.generate(**inputs, max_new_tokens=250)

    result = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # ✂️ Clean result: remove prompt from model output if it repeats
    if result.lower().startswith(prompt.lower().split(" Keep")[0]):
        result = result[len(prompt.split(" Keep")[0]):].strip()

    # ✅ Optional: only keep first 3 tips (in case model gives more)
    tips = []
    for line in result.split('\n'):
        if line.strip().startswith(('1', '2', '3')):
            tips.append(line.strip())
        if len(tips) == 3:
            break
    result_text = "\n".join(tips)

    return f"Prompt: {prompt.split(' Keep')[0]}\n\n{result_text}"


# 🔸 Feedback Form
feedback_storage = []

def submit_feedback(name, email, message):
    # Add new feedback to session list
    feedback_storage.append((name, message))

    # Create a display string from all feedbacks
    all_feedbacks = "\n".join([f"{n} - {m}" for n, m in feedback_storage])

    return f"✅ Feedback submitted!\n\n📋 All Feedbacks This Session:\n{all_feedbacks}"



# 🔸 PDF Report Generator
def generate_report(content):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    # Split by paragraphs or lines
    for line in content.split('\n'):
        pdf.multi_cell(0, 10, txt=line.strip(), align='L')
        # pdf.ln()

    file_path = "/tmp/smart_city_report.pdf"
    pdf.output(file_path)
    return file_path

# New functionalities

from sklearn.linear_model import LinearRegression
import pandas as pd

# 🔸 KPI Forecasting
def forecast_kpi(file, kpi_column):
    df = pd.read_csv(file.name)
    if 'year' not in df.columns or kpi_column not in df.columns:
        return "❌ CSV must contain 'year' and the selected KPI column."

    model = LinearRegression()
    X = df[['year']]
    y = df[[kpi_column]]
    model.fit(X, y)

    next_year = [[df['year'].max() + 1]]
    prediction = model.predict(next_year)[0][0]
    return f"📈 Forecast for {kpi_column} in {int(next_year[0][0])}: {prediction:.2f}"

# 🔸 Anomaly Detection
def detect_anomalies(file, kpi_column, threshold):
    df = pd.read_csv(file.name)
    if kpi_column not in df.columns:
        return "❌ Selected KPI not found in the uploaded CSV."

    anomalies = df[df[kpi_column] > threshold]
    if anomalies.empty:
        return "✅ No anomalies found above the threshold."

    return f"🚨 {len(anomalies)} anomalies found above {threshold}:\n\n" + anomalies.to_string(index=False)



In [6]:
# UI

import gradio as gr

# 🔹 Chat Tab
chat_ui = gr.Interface(
    fn=smart_city_chat,
    inputs=gr.Textbox(placeholder="Ask your smart city assistant..."),
    outputs="text",
    title="🤖 Smart City Chatbot"
)

# 🔹 PDF Summarizer
pdf_ui = gr.Interface(
    fn=summarize_pdf,
    inputs=gr.File(file_types=[".pdf"]),
    outputs="text",
    title="📄 Policy Summarizer"
)

# 🔹 KPI Dashboard
kpi_ui = gr.Interface(
    fn=kpi_summary,
    inputs=[],
    outputs="markdown",
    title="📊 KPI Dashboard"
)

# 🔹 Eco Tips
eco_ui = gr.Interface(
    fn=generate_eco_tips,
    inputs=[],
    outputs="text",
    title="🌱 Eco Advice"
)

# 🔹 Feedback Form
feedback_ui = gr.Interface(
    fn=submit_feedback,
    inputs=[
        gr.Textbox(label="Name"),
        gr.Textbox(label="Email"),
        gr.Textbox(label="Feedback", lines=3)
    ],
    outputs="text",
    title="📝 Feedback"
)


# 🔹 Report Generator
report_ui = gr.Interface(
    fn=generate_report,
    inputs=gr.Textbox(label="Report Content", lines=8),
    outputs="file",
    title="🖨️ Generate Report PDF"
)


# 🔹 KPI Forecasting Tab
forecast_ui = gr.Interface(
    fn=forecast_kpi,
    inputs=[
        gr.File(label="Upload KPI CSV"),
        gr.Textbox(label="Enter KPI column (e.g., energy)")
    ],
    outputs="text",
    title="📈 KPI Forecasting"
)

# 🔹 Anomaly Detection Tab
anomaly_ui = gr.Interface(
    fn=detect_anomalies,
    inputs=[
        gr.File(label="Upload KPI CSV"),
        gr.Textbox(label="Enter KPI column (e.g., energy)"),
        gr.Number(label="Set Threshold", value=1000.0)
    ],
    outputs="text",
    title="🚨 Anomaly Detection"
)



# 🔹 Combine Tabs
gr.TabbedInterface(
    [chat_ui, pdf_ui, kpi_ui, forecast_ui, anomaly_ui, eco_ui, feedback_ui, report_ui],
    ["Chat", "Policy Summary", "KPI Cards", "KPI Forecasting", "Anomaly Detection", "Eco Tips", "Feedback", "Report Generator"]
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd9a1055de440c5562.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
